In [1]:
# -*- coding: utf-8 -*-
# --------------------------------------------------
#
# CRF4.ipynb
#
# (1) Token lemmatization using Zemberek TurkishMorphology  (DO NOT lemmatize tokens starting with uppercase letter)
# (2) Token features:
#     (a) token
#     (b) is_first           : is token at the beginning of the sentence?
#     (c) is_last            : is token at the end of the sentence?
#     (d) is_capitalized     : does token start with a capital letter? 
#     (e) is_all_capitalized : is all letters of the token capitalized?
#     (f) is_capitals_inside : is there any capitalized letter inside the token?
#     (g) is_numeric         : is token numeric?
#     (h) prefix-1           : first letter of the token
#     (i) suffix-1           : last letter of the token
#     (j) prefix-2           : first two letters of the token
#     (k) suffix-2           : last two letters of the token
#
# Written by cetinsamet -*- cetin.samet@metu.edu.tr
# May, 2019
# --------------------------------------------------

In [2]:
from sklearn.model_selection import RandomizedSearchCV
from seqeval.metrics import classification_report
from sklearn.metrics import make_scorer
from sklearn_crfsuite import metrics
from sklearn_crfsuite import CRF
import jpype as jp
import pickle
import scipy

In [3]:
ZEMBEREK_PATH = 'bin/zemberek-full.jar'

# Start the JVM
jp.startJVM(jp.getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

In [4]:
TurkishMorphology = jp.JClass('zemberek.morphology.TurkishMorphology')
morphology        = TurkishMorphology.createWithDefaults()

In [5]:
def readFile(filepath):

    text     = []
    sentence = []

    with open(filepath, 'r') as infile:
        for line in infile:
            word, _, _, _ = line.strip().split('\t')

            if word == '<S>':
                text.append(sentence)
                sentence = []
                continue

            sentence.append(line.strip())

    return text

In [6]:
trainText = readFile('data/train.txt')
validText = readFile('data/valid.txt')
testText  = readFile('data/test.txt')

In [8]:
def getFeature(token, token_index, sentence):

    feature = {'token'             : token,
               'is_first'          : token_index == 0,
               'is_last'           : token_index == len(sentence) - 1,
               
               'is_capitalized'    : token[0].upper() == token[0],
               'is_all_capitalized': token.upper() == token,
               'is_capitals_inside': token[1:].lower() != token[1:],
               'is_numeric'        : token.isdigit(),
               
               'prefix-1'          : token[0],
               'suffix-1'          : token[-1],
               
               'prefix-2'          : '' if len(token) < 2  else token[:2],
               'suffix-2'          : '' if len(token) < 2  else token[-2:]
              }
    
    return feature

In [9]:
trainFeatures = []
trainLabels   = []

for sentence in trainText:
    SENT     = []
    features = []
    labels   = []
    for token in sentence:
        word, _, _, label = token.split('\t')
        
        if word[0] == word[0].upper():
            root = word
        else:
            results = morphology.analyze(word).analysisResults
            root    = results[0].getLemmas()[0] if results else word
        SENT.append(root)
        labels.append(label)
        
    for i, word in enumerate(SENT):
        features.append(getFeature(word, i, SENT))
    
    trainFeatures.append(features)
    trainLabels.append(labels)

In [10]:
validFeatures = []
validLabels   = []

for sentence in validText:
    SENT     = []
    features = []
    labels   = []
    for token in sentence:
        word, _, _, label = token.split('\t')
        
        if word[0] == word[0].upper():
            root = word
        else:
            results = morphology.analyze(word).analysisResults
            root    = results[0].getLemmas()[0] if results else word
        SENT.append(root)
        labels.append(label)
        
    for i, word in enumerate(SENT):
        features.append(getFeature(word, i, SENT))
    
    validFeatures.append(features)
    validLabels.append(labels)

In [11]:
testFeatures = []
testLabels   = []

for sentence in testText:
    SENT     = []
    features = []
    labels   = []
    for token in sentence:
        word, _, _, label = token.split('\t')
        
        if word[0] == word[0].upper():
            root = word
        else:
            results = morphology.analyze(word).analysisResults
            root    = results[0].getLemmas()[0] if results else word
        SENT.append(root)
        labels.append(label)
        
    for i, word in enumerate(SENT):
        features.append(getFeature(word, i, SENT))
    
    testFeatures.append(features)
    testLabels.append(labels)

In [12]:
trainvalFeatures = trainFeatures + validFeatures
trainvalLabels   = trainLabels   + validLabels

In [13]:
# define fixed parameters and parameters to search
crf = CRF(  algorithm='lbfgs',
            max_iterations=100,
            all_possible_transitions=True,
            verbose=True)

params_space = {'c1': scipy.stats.expon(scale=0.5),
                'c2': scipy.stats.expon(scale=0.05)}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        random_state=123,
                        scoring=f1_scorer)

rs.fit(trainvalFeatures, trainvalLabels)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 36.3min
/Users/cetinsamet/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 150.3min finished
loading training data to CRFsuite: 100%|██████████| 32171/32171 [00:04<00:00, 8003.37it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 43164
Seconds required: 0.865

L-BFGS optimization
c1: 0.008130
c2: 0.045123
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=1.02  loss=796297.69 active=43121 feature_norm=1.00
Iter 2   time=2.12  loss=494833.41 active=42990 feature_norm=5.94
Iter 3   time=0.50  loss=397707.59 active=40923 feature_norm=5.07
Iter 4   time=2.55  loss=261697.25 active=41982 feature_norm=3.74
Iter 5   time=0.98  loss=253601.58 active=42796 feature_norm=3.74
Iter 6   time=0.49  loss=225202.88 active=42994 feature_norm=4.35
Iter 7   time=0.49  loss=192721.68 active=43006 feature_norm=5.69
Iter 8   time=0.49  loss=150523.33 active=42363 feature_norm=9.40
Iter 9   time=0.49  loss=137388.44 active=42740 feature_norm=11.06
Iter

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=None, c2=None,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error...ne,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True),
          fit_params=None, iid='warn', n_iter=50, n_jobs=-1,
          param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a18f6b588>, 'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1afa527e10>},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn',
          scoring=make_scorer(flat_f1_score, average=weighted), verbose=1)

In [16]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.008130349084185253, 'c2': 0.04512332140157257}
best CV score: 0.9624091658062226
model size: 2.04M


In [15]:
crf = rs.best_estimator_
crf

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None,
  c1=0.008130349084185253, c2=0.04512332140157257,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True)

In [17]:
crf.fit(trainvalFeatures, trainvalLabels)

loading training data to CRFsuite: 100%|██████████| 32171/32171 [00:03<00:00, 8093.54it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 43164
Seconds required: 0.827

L-BFGS optimization
c1: 0.008130
c2: 0.045123
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.93  loss=796297.69 active=43121 feature_norm=1.00
Iter 2   time=1.89  loss=494833.41 active=42990 feature_norm=5.94
Iter 3   time=0.47  loss=397707.59 active=40923 feature_norm=5.07
Iter 4   time=2.52  loss=261697.25 active=41982 feature_norm=3.74
Iter 5   time=0.95  loss=253601.58 active=42796 feature_norm=3.74
Iter 6   time=0.51  loss=225202.88 active=42994 feature_norm=4.35
Iter 7   time=0.62  loss=192721.68 active=43006 feature_norm=5.69
Iter 8   time=0.58  loss=150523.33 active=42363 feature_norm=9.40
Iter 9   time=0.57  loss=137388.44 active=42740 feature_norm=11.06
Iter

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None,
  c1=0.008130349084185253, c2=0.04512332140157257,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True)

In [18]:
# SAVE CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf4.pickle', 'wb') as outfile:
    pickle.dump(crf, outfile, pickle.HIGHEST_PROTOCOL)
    print("model is saved.")

model is saved.


In [19]:
# LOAD CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf4.pickle', 'rb') as infile:
    crf = pickle.load(infile)

In [20]:
trainvalPredLabels = crf.predict(trainvalFeatures)

print("### TRAINVAL CLASSIFICATION REPORT ###\n")
print(classification_report(trainvalLabels, trainvalPredLabels))

### TRAINVAL CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

     PERCENT       1.00      1.00      1.00       617
ORGANIZATION       0.97      0.96      0.97      9034
      PERSON       0.98      0.98      0.98     14476
        DATE       0.78      0.77      0.77      3103
    LOCATION       0.98      0.99      0.98      9409
       MONEY       0.97      0.94      0.96       594
        TIME       0.99      0.99      0.99       175

   micro avg       0.96      0.96      0.96     37408
   macro avg       0.96      0.96      0.96     37408



In [21]:
testPredLabels  = crf.predict(testFeatures)

print("### TEST CLASSIFICATION REPORT ###\n")
print(classification_report(testLabels, testPredLabels))

### TEST CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

ORGANIZATION       0.81      0.77      0.79       862
    LOCATION       0.88      0.80      0.84      1091
      PERSON       0.88      0.80      0.84      1594
       MONEY       0.87      0.73      0.80       113
        DATE       0.71      0.72      0.72       364
     PERCENT       0.98      0.93      0.96       107
        TIME       0.78      0.78      0.78        23

   micro avg       0.85      0.79      0.82      4154
   macro avg       0.85      0.79      0.82      4154



In [22]:
# Shutting down the JVM
jp.shutdownJVM()